In [ ]:
! ray stop && ray start --head --disable-usage-stats

In [2]:
import ray
import time
import asyncio
from ray import serve
import pandas as pd
import mlflow.pyfunc
import mlflow
import os
from app.main import app
from app.features.model.crud import repo as models_repo
from app.db.session import SessionLocal
from app.features.model.schema.model import Model

if ray.is_initialized:
    ray.shutdown()

address = 'auto'
ray.init(address=address)

serve.start(detached=True)

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
File descriptor limit 1024 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'


In [3]:
@serve.deployment
class MLflowDeployment:
    def __init__(self, model_uri):
        self.model = mlflow.pyfunc.load_model(model_uri=model_uri)

    async def __call__(self, request):
        csv_text = await request.body() # The body contains just raw csv text.
        df = pd.read_csv(csv_text)
        return self.model.predict(df)

def deploy_model(deployment_name: str, model: Model, route_prefix: str):
    MLflowDeployment.options(name=deployment_name, route_prefix=route_prefix).deploy(model.get_model_uri())


In [4]:
## Get a model from db
db = SessionLocal()
db_models, total = models_repo.get_paginated(db, page=0, per_page=15)
model = Model.from_orm(db_models[0])
## Deploy the model
print(model.get_model_uri())
deployment_name = f'{model.name}-deployment'
route_prefix = '/example-deployment2'
deploy_model(deployment_name, model, route_prefix)

model:/grsaeoohffsxkedcunlrxjposjwzagcx/1


RuntimeError: Deployment grsaeoohffsxkedcunlrxjposjwzagcx-deployment is UNHEALTHY: The Deployment constructor failed 3 times in a row. See logs for details.